In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import pyarrow.dataset as ds
import duckdb

pd.set_option('display.max_columns', None)

In [ ]:
candlestick_path = "data/local_data/candlesticks/"

In [ ]:
dataset = ds.dataset(candlestick_path, format="parquet")
len(dataset.files)

In [ ]:
dataset.files[0]

In [ ]:
df = pd.read_parquet(dataset.files[2000])

In [ ]:
df = df.reset_index()

In [ ]:
# The difference in time between the two records grouped by 'market_ticker'
df["time_delta"] = df["end_period_ts"] - df.groupby("market_ticker")["end_period_ts"].shift(1)

In [ ]:
df[["market_ticker", "end_period_ts", "time_delta"]].head(20)

In [ ]:
df.columns

In [ ]:
1761097140 - 1761097080

# Read all via DuckDB

In [ ]:
con = duckdb.connect()

con.execute(
    """
    SELECT *
    FROM read_parquet('data/local_data/candlesticks/*.parquet');
    """
)

df_candlesticks = con.fetch_df()

In [ ]:
df_candlesticks.shape

In [ ]:
df_candlesticks.head()

In [ ]:
df_candlesticks.tail()

In [ ]:
df_candlesticks.columns

In [ ]:
df_candlesticks = df_candlesticks[
    [
        "market_ticker",
        "end_period_ts",
        "volume",
        "open_interest",
        "yes_ask_close_dollars",
        "yes_ask_high_dollars",
        "yes_ask_low_dollars",
        "yes_ask_open_dollars",
        "yes_bid_close_dollars",
        "yes_bid_high_dollars",
        "yes_bid_low_dollars",
        "yes_bid_open_dollars",
        "price_close_dollars",
        "price_high_dollars",
        "price_low_dollars",
        "price_mean_dollars",
        "price_open_dollars",
        "price_previous_dollars",
    ]
]

In [ ]:
df_candlesticks.head()

In [ ]:
df_candlesticks["market_ticker"].nunique()

In [ ]:
df_candlesticks["series_ticker"] = df_candlesticks["market_ticker"].str.split("-").str[0]
df_candlesticks["series_ticker"].nunique()

In [ ]:
df_candlesticks["date_str"] = df_candlesticks["market_ticker"].str.split("-").str[1]
df_candlesticks["temp_str"] = df_candlesticks["market_ticker"].str.split("-").str[2]

In [ ]:
df_candlesticks["temp_type"] = df_candlesticks["temp_str"].str[0]
df_candlesticks["temp_value"] = df_candlesticks["temp_str"].str[1:]

In [ ]:
df_candlesticks["temp_type"].value_counts()

In [ ]:
df_candlesticks["temp_value"].value_counts()

In [ ]:
df_candlesticks.head()

In [ ]:
market_temp_details = df_candlesticks.pivot_table(
    index=["series_ticker", "date_str", "temp_type", "temp_value"],
    values="end_period_ts",
    aggfunc="min",
).reset_index()

In [ ]:
market_temp_details = market_temp_details.sort_values(by=["series_ticker", "date_str", "temp_value"])

In [ ]:
market_temp_details["strike_rank"] = market_temp_details.groupby(["series_ticker", "date_str"]).cumcount() + 1

In [ ]:
market_temp_details["strike_rank"].value_counts()

In [ ]:
market_temp_details.head(6)

In [ ]:
df_candlesticks = df_candlesticks.merge(
    market_temp_details[
        ["series_ticker", "date_str", "temp_type", "temp_value", "strike_rank"]
    ],
    how="left",
    on=["series_ticker", "date_str", "temp_type", "temp_value"],
)

In [ ]:
df_candlesticks.head()

In [ ]:
df_candlesticks["event_ticker"] = df_candlesticks["series_ticker"].astype(str) + "-" + df_candlesticks["date_str"].astype(str)

In [ ]:
df_candlesticks["event_ticker"].value_counts()

In [ ]:
df_candlesticks[["price_close_dollars"]].info()

In [ ]:
df_candlesticks["price_close_dollars"] = df_candlesticks["price_close_dollars"].astype(float)

In [ ]:
values = ["price_close_dollars", "volume", "open_interest"]
for value in values:
    display(
        df_candlesticks[
            (df_candlesticks["event_ticker"] == "KXHIGHAUS-25OCT23")
            & (df_candlesticks["open_interest"] > 0)
            & (df_candlesticks["volume"] > 0)
        ].pivot_table(
            index=["strike_rank", "temp_str"],
            # columns="strike_rank",
            values=value,
            aggfunc=["max", "min", "mean", "median"],
        ).reset_index()
    )

NEXT STEPS:

Add a plot of each of these values over time throughout the day.
